## frist time install

In [2]:
# pip install gdown
#!pip install pandas
#!pip install matplotlib

## import library

In [3]:
import glob
import pandas as pd

import os
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import re

## Import data

In [12]:
!gdown --id 1Fq7KX6xy-Nk6ZI6qEVyRJNpGbLjXq_vD

Downloading...
From: https://drive.google.com/uc?id=1Fq7KX6xy-Nk6ZI6qEVyRJNpGbLjXq_vD
To: /root/LST20.tar.gz
100%|███████████████████████████████████████| 12.3M/12.3M [00:00<00:00, 294MB/s]


In [ ]:
!tar -xzvf "LST20.tar.gz"

In [4]:
train = glob.glob('LST20_Corpus/train/*.txt')
val = glob.glob('LST20_Corpus/eval/*.txt')
test = glob.glob('LST20_Corpus/test/*.txt')
len(train), len(val), len(test)

(3794, 474, 483)

# Prepare data [Train]

1. Label "O" to blank word
2. Change CLS to SENT
3. Check error "OO" > "IO"

## Import data

* Split data

In [209]:
li = []

for filename in train:
    f = open(filename).read()
    f = list(map(lambda x:x.split('\t'),f.split('\n')))
    # print(f)
    del f[-1]
    li+=f
    #print(len(li))

In [207]:
#Try
li[0:2]

[['"', 'PU', 'O', 'B_CLS'], ['สมชาย', 'NN', 'B_PER', 'I_CLS']]

In [217]:
#Try
len(li)

2778036

if want combile "val & Test" in li

In [460]:
# for filename in val:
#     f = open(filename).read()
#     f = list(map(lambda x:x.split('\t'),f.split('\n')))
#     #print(f)
#     del f[-1]
#     li+=f
#     #print(len(li))

# for filename in test:
#     f = open(filename).read()
#     f = list(map(lambda x:x.split('\t'),f.split('\n')))
#     #print(f)
#     del f[-1]
#     li+=f
#     #print(len(li))

* Create data frame

In [304]:
df = pd.DataFrame(columns = ['word', 'pos', 'ner', 'clause'], data = [li][0][:])
df

,word,pos,ner,clause
0,"""",PU,O,B_CLS
1,สมชาย,NN,B_PER,I_CLS
2,"""",PU,O,I_CLS
3,ลอยลำ,VV,O,I_CLS
4,นายก,NN,O,I_CLS
...,...,...,...,...
2778031,_,PU,I_MEA,I_CLS
2778032,บาท,CL,E_MEA,I_CLS
2778033,_,PU,O,I_CLS
2778034,เป็นต้น,AV,O,E_CLS


* Recheck

In [305]:
#count
df.count()

word      2778036
pos       2714726
ner       2714726
clause    2714726
dtype: int64

In [306]:
#count word "" = sentence // True word: 2778036-63310=2714726
df.groupby([df["word"] == ""]).count()

,word,pos,ner,clause
word,,,,
False,2714726,2714726,2714726,2714726
True,63310,0,0,0


**Check Point [df1]**

In [324]:
df1=df.copy()
#try
df1

,word,pos,ner,clause
0,"""",PU,O,B_CLS
1,สมชาย,NN,B_PER,I_CLS
2,"""",PU,O,I_CLS
3,ลอยลำ,VV,O,I_CLS
4,นายก,NN,O,I_CLS
...,...,...,...,...
2778031,_,PU,I_MEA,I_CLS
2778032,บาท,CL,E_MEA,I_CLS
2778033,_,PU,O,I_CLS
2778034,เป็นต้น,AV,O,E_CLS


## Label "O"

original file have "O" > Remove "O" to "delete"

In [325]:
#original [df]
df[df["clause"] == "O"].count()

word      74079
pos       74079
ner       74079
clause    74079
dtype: int64

In [326]:
#original [df]
df[df["word"] == "_"].count()

word      402380
pos       402380
ner       402380
clause    402380
dtype: int64

In [327]:
#original [df]
df[df["word"] == ""].count()

word      63310
pos           0
ner           0
clause        0
dtype: int64

In [328]:
#copy [df1]
df1.clause[df1["clause"] == "O"] = "delete" #O=Space
df1[df1["clause"] == "delete"]

,word,pos,ner,clause
157,_,PU,O,delete
186,_,PU,O,delete
190,_,PU,O,delete
258,_,PU,O,delete
284,_,PU,O,delete
...,...,...,...,...
2777906,_,PU,O,delete
2777914,_,PU,O,delete
2777966,_,PU,O,delete
2777973,_,PU,O,delete


* label "O" in word " "

In [329]:
df1.pos[df1["word"] == ""] = "XX" #XX=other
df1.ner[df1["word"] == ""] = "O" #O=outside
df1.clause[df1["word"] == ""] = "O" #O=Space
df1.word[df1["word"] == ""] = " " # space
df1[df1["word"] == " "]

,word,pos,ner,clause
15,,XX,O,O
36,,XX,O,O
76,,XX,O,O
86,,XX,O,O
152,,XX,O,O
...,...,...,...,...
2777829,,XX,O,O
2777897,,XX,O,O
2777945,,XX,O,O
2777990,,XX,O,O


* change space word "_" to word " "

In [330]:
df1.word[df1["word"] == "_"] = " " #change word "_" to " "
#check
df1.word[df1["word"] == "_"]

Series([], Name: word, dtype: object)

In [331]:
#Cheack clause
df1.groupby("clause").count()

,word,pos,ner
clause,,,
B_CLS,215301,215301,215301
E_CLS,213247,213247,213247
I_CLS,2212099,2212099,2212099
O,63310,63310,63310
delete,74079,74079,74079


* change label "delete" to "I_CLS"

In [332]:
df1.clause[df1["clause"] == "delete"] = "I_CLS" #O=Space
#check
df1.clause[df1["clause"] == "delete"]

Series([], Name: clause, dtype: object)

In [333]:
#Cheack clause
df1.groupby("clause").count()

,word,pos,ner
clause,,,
B_CLS,215301,215301,215301
E_CLS,213247,213247,213247
I_CLS,2286178,2286178,2286178
O,63310,63310,63310


## Save Original_Clean_Train.csv

In [334]:
df1.to_csv('Original_Clean_Train.csv')

**Check point [df_SENT]**

In [335]:
df_SENT=df1.copy()
#try
df_SENT

,word,pos,ner,clause
0,"""",PU,O,B_CLS
1,สมชาย,NN,B_PER,I_CLS
2,"""",PU,O,I_CLS
3,ลอยลำ,VV,O,I_CLS
4,นายก,NN,O,I_CLS
...,...,...,...,...
2778031,,PU,I_MEA,I_CLS
2778032,บาท,CL,E_MEA,I_CLS
2778033,,PU,O,I_CLS
2778034,เป็นต้น,AV,O,E_CLS


In [336]:
#Cheack clause
df_SENT.groupby("clause").count()

,word,pos,ner
clause,,,
B_CLS,215301,215301,215301
E_CLS,213247,213247,213247
I_CLS,2286178,2286178,2286178
O,63310,63310,63310


## Change B,I,E_CLS to I_SENT

In [337]:
df_SENT.clause[df_SENT["clause"] == "B_CLS"] = "I_SENT"
df_SENT.clause[df_SENT["clause"] == "I_CLS"] = "I_SENT"
df_SENT.clause[df_SENT["clause"] == "E_CLS"] = "I_SENT"
#try
df_SENT

,word,pos,ner,clause
0,"""",PU,O,I_SENT
1,สมชาย,NN,B_PER,I_SENT
2,"""",PU,O,I_SENT
3,ลอยลำ,VV,O,I_SENT
4,นายก,NN,O,I_SENT
...,...,...,...,...
2778031,,PU,I_MEA,I_SENT
2778032,บาท,CL,E_MEA,I_SENT
2778033,,PU,O,I_SENT
2778034,เป็นต้น,AV,O,I_SENT


In [338]:
#Cheack clause
df_SENT.groupby("clause").count()

,word,pos,ner
clause,,,
I_SENT,2714726,2714726,2714726
O,63310,63310,63310


## Save IO_Train.csv

In [339]:
df_SENT.to_csv('IO_Train.csv')

**Check point [df_SENT2]**

In [340]:
df_SENT2=df_SENT.copy()
#try
df_SENT2

,word,pos,ner,clause
0,"""",PU,O,I_SENT
1,สมชาย,NN,B_PER,I_SENT
2,"""",PU,O,I_SENT
3,ลอยลำ,VV,O,I_SENT
4,นายก,NN,O,I_SENT
...,...,...,...,...
2778031,,PU,I_MEA,I_SENT
2778032,บาท,CL,E_MEA,I_SENT
2778033,,PU,O,I_SENT
2778034,เป็นต้น,AV,O,I_SENT


## --Check error OO

Not have

In [341]:
# Find index of "O"
index_O=df_SENT[(df_SENT['clause']=='O')].index
index_O

Int64Index([     15,      36,      76,      86,     152,     222,     294,
                356,     427,     462,
            ...
            2777591, 2777646, 2777701, 2777735, 2777823, 2777829, 2777897,
            2777945, 2777990, 2778035],
           dtype='int64', length=63310)

* count OO, OOO

In [344]:
error_OO=[]
error_OOO=[]
for n in index_O:
    try:
        if df_SENT2["clause"][n+2]=='O':
            if df_SENT2["clause"][n+1]=='O':
                error_OOO.append(n)
        elif df_SENT2["clause"][n+1]=='O':
            error_OO.append(n)
    except:
        print("Finish")

error_OO,error_OOO
# n=error_OO[0]
# print("Check",n)
# print(df_SENT2["clause"][n-1],"-",df_SENT2["clause"][n],"-",df_SENT2["clause"][n+1],"-",df_SENT2["clause"][n+2])
# print(df_SENT2["word"][n-1],"-",df_SENT2["word"][n],"-",df_SENT2["word"][n+1],"-",df_SENT2["word"][n+2])

Finish


([], [])

In [345]:
# Change OO to IO
for m in error_OO:
    df_SENT2["clause"][m]= "I_SENT"
#try
df_SENT2.groupby("clause").count()

,word,pos,ner
clause,,,
I_SENT,2714726,2714726,2714726
O,63310,63310,63310


## -- Save IO2_Train.csv

In [347]:
# df_SENT2.to_csv('IO2_Train.csv') #Not have OO error

**Check point [df_SENT3]**

In [349]:
df_SENT3=df_SENT2.copy()
#try
df_SENT3

,word,pos,ner,clause
0,"""",PU,O,I_SENT
1,สมชาย,NN,B_PER,I_SENT
2,"""",PU,O,I_SENT
3,ลอยลำ,VV,O,I_SENT
4,นายก,NN,O,I_SENT
...,...,...,...,...
2778031,,PU,I_MEA,I_SENT
2778032,บาท,CL,E_MEA,I_SENT
2778033,,PU,O,I_SENT
2778034,เป็นต้น,AV,O,I_SENT


## -- Rule Base: E,O,B

In [350]:
# Find index of "O"
index_O3=df_SENT3[(df_SENT3['clause']=='O')].index
index_O3

Int64Index([     15,      36,      76,      86,     152,     222,     294,
                356,     427,     462,
            ...
            2777591, 2777646, 2777701, 2777735, 2777823, 2777829, 2777897,
            2777945, 2777990, 2778035],
           dtype='int64', length=63310)

In [352]:
#fill E,B_SENT
df_SENT3.loc[index_O3-1, "clause"] = "E_SENT"
df_SENT3.loc[index_O3[:-1]+1, "clause"] = "B_SENT"
#Cheack clause
df_SENT3.groupby("clause").count()

,word,pos,ner
clause,,,
B_SENT,63309,63309,63309
E_SENT,63202,63202,63202
I_SENT,2588215,2588215,2588215
O,63310,63310,63310


## -- Save BIEO_Train.csv

In [353]:
df_SENT3.to_csv('BIEO_Train.csv')

**Check point [df_SENT_BIEO]**

In [354]:
df_SENT_BIEO=df_SENT3.copy()
#try
df_SENT_BIEO

,word,pos,ner,clause
0,"""",PU,O,I_SENT
1,สมชาย,NN,B_PER,I_SENT
2,"""",PU,O,I_SENT
3,ลอยลำ,VV,O,I_SENT
4,นายก,NN,O,I_SENT
...,...,...,...,...
2778031,,PU,I_MEA,I_SENT
2778032,บาท,CL,E_MEA,I_SENT
2778033,,PU,O,I_SENT
2778034,เป็นต้น,AV,O,E_SENT


## Clean Data

delete 2,<=400 sentences

* Count words

In [355]:
df_SENT_BIEO.describe()

,word,pos,ner,clause
count,2778036,2778036,2778036,2778036
unique,48945,16,40,4
top,,NN,O,I_SENT
freq,465690,758019,2322403,2588215


* Tracking sentence

In [359]:
def convert_to_simple_transformer_format(df):
    word = []
    pos = []
    ner = []
    clause = []
    sentence_id = []
    current_sentence = 1
    
    for i in range(len(df['clause'])):
        sentence_id.append(current_sentence)
        word.append(df['word'][i])
        pos.append(df['pos'][i])
        ner.append(df['ner'][i])
        clause.append(df['clause'][i])
        if(df['clause'][i] == 'O'):
            current_sentence = current_sentence + 1

    return pd.DataFrame(
        {"word": word, "pos":pos,"ner":ner, "clause": clause,"sentence_id": sentence_id}
    )

In [363]:
df_SENT_BIEO2=convert_to_simple_transformer_format(df_SENT_BIEO)
df_SENT_BIEO2 #// True 63,310 sentence

,word,pos,ner,clause,sentence_id
0,"""",PU,O,I_SENT,1
1,สมชาย,NN,B_PER,I_SENT,1
2,"""",PU,O,I_SENT,1
3,ลอยลำ,VV,O,I_SENT,1
4,นายก,NN,O,I_SENT,1
...,...,...,...,...,...
2778031,,PU,I_MEA,I_SENT,63310
2778032,บาท,CL,E_MEA,I_SENT,63310
2778033,,PU,O,I_SENT,63310
2778034,เป็นต้น,AV,O,E_SENT,63310


**Check point [df_SENT_BIEO3]**

In [435]:
df_SENT_BIEO3=df_SENT_BIEO2.copy()
#try
df_SENT_BIEO3

,word,pos,ner,clause,sentence_id
0,"""",PU,O,I_SENT,1
1,สมชาย,NN,B_PER,I_SENT,1
2,"""",PU,O,I_SENT,1
3,ลอยลำ,VV,O,I_SENT,1
4,นายก,NN,O,I_SENT,1
...,...,...,...,...,...
2778031,,PU,I_MEA,I_SENT,63310
2778032,บาท,CL,E_MEA,I_SENT,63310
2778033,,PU,O,I_SENT,63310
2778034,เป็นต้น,AV,O,E_SENT,63310


* delete word = 2

In [437]:
#find id > word=2
count_BIEO3=df_SENT_BIEO3.groupby("sentence_id").count()
sentence_id2=count_BIEO3[count_BIEO3['word']== 2].index
sentence_id2

Int64Index([12375, 14022, 17482, 20389, 21303, 21304, 21305, 21306, 21307,
            21308,
            ...
            55204, 55206, 55207, 55210, 56800, 58635, 58719, 59366, 59367,
            62781],
           dtype='int64', name='sentence_id', length=108)

In [436]:
#Try
df_SENT_BIEO3['word'][546871]

"Office of His Majesty's Principal Private Secretary"

In [438]:
for i in sentence_id2:
    index_sentence_id2=df_SENT_BIEO3[df_SENT_BIEO3["sentence_id"]== i].index
    df_SENT_BIEO3=df_SENT_BIEO3.drop(index_sentence_id2)
df_SENT_BIEO3.reset_index(inplace=True)
df_SENT_BIEO3

,index,word,pos,ner,clause,sentence_id
0,0,"""",PU,O,I_SENT,1
1,1,สมชาย,NN,B_PER,I_SENT,1
2,2,"""",PU,O,I_SENT,1
3,3,ลอยลำ,VV,O,I_SENT,1
4,4,นายก,NN,O,I_SENT,1
...,...,...,...,...,...,...
2777815,2778031,,PU,I_MEA,I_SENT,63310
2777816,2778032,บาท,CL,E_MEA,I_SENT,63310
2777817,2778033,,PU,O,I_SENT,63310
2777818,2778034,เป็นต้น,AV,O,E_SENT,63310


In [439]:
#Try
df_SENT_BIEO3['word'][546871]

'ข่าว'

* delete word >= 400

In [440]:
#find id > word>=400
count_BIEO3=df_SENT_BIEO3.groupby("sentence_id").count()
sentence_id400=count_BIEO3[count_BIEO3['word']>= 400].index
sentence_id400

Int64Index([62, 9810, 21214, 47132, 47133, 47336, 51119], dtype='int64', name='sentence_id')

In [442]:
for i in sentence_id400:
    index_sentence_id400=df_SENT_BIEO3[df_SENT_BIEO3["sentence_id"]== i].index
    df_SENT_BIEO3=df_SENT_BIEO3.drop(index_sentence_id400)
df_SENT_BIEO3.reset_index(inplace=True)
df_SENT_BIEO3

,level_0,index,word,pos,ner,clause,sentence_id
0,0,0,"""",PU,O,I_SENT,1
1,1,1,สมชาย,NN,B_PER,I_SENT,1
2,2,2,"""",PU,O,I_SENT,1
3,3,3,ลอยลำ,VV,O,I_SENT,1
4,4,4,นายก,NN,O,I_SENT,1
...,...,...,...,...,...,...,...
2773709,2777815,2778031,,PU,I_MEA,I_SENT,63310
2773710,2777816,2778032,บาท,CL,E_MEA,I_SENT,63310
2773711,2777817,2778033,,PU,O,I_SENT,63310
2773712,2777818,2778034,เป็นต้น,AV,O,E_SENT,63310


## Save Data_Train.csv

In [446]:
df_SENT_BIEO4 = pd.DataFrame(columns = ['word', 'pos', 'ner', 'clause'], data = df_SENT_BIEO3)
df_SENT_BIEO4

,word,pos,ner,clause
0,"""",PU,O,I_SENT
1,สมชาย,NN,B_PER,I_SENT
2,"""",PU,O,I_SENT
3,ลอยลำ,VV,O,I_SENT
4,นายก,NN,O,I_SENT
...,...,...,...,...
2773709,,PU,I_MEA,I_SENT
2773710,บาท,CL,E_MEA,I_SENT
2773711,,PU,O,I_SENT
2773712,เป็นต้น,AV,O,E_SENT


In [447]:
df_SENT_BIEO4.to_csv('Data_Train.csv')

**Check point [df_Train]**

In [448]:
df_Train=df_SENT_BIEO4.copy()
#try
df_Train

,word,pos,ner,clause
0,"""",PU,O,I_SENT
1,สมชาย,NN,B_PER,I_SENT
2,"""",PU,O,I_SENT
3,ลอยลำ,VV,O,I_SENT
4,นายก,NN,O,I_SENT
...,...,...,...,...
2773709,,PU,I_MEA,I_SENT
2773710,บาท,CL,E_MEA,I_SENT
2773711,,PU,O,I_SENT
2773712,เป็นต้น,AV,O,E_SENT


* Split data

In [461]:
li = []

for filename in train:
    f = open(filename).read()
    f = list(map(lambda x:x.split('\t'),f.split('\n')))
    # print(f)
    del f[-1]
    li+=f
    #print(len(li))

In [462]:
#Try
li[0:2]

[['"', 'PU', 'O', 'B_CLS'], ['สมชาย', 'NN', 'B_PER', 'I_CLS']]

In [463]:
#Try
len(li)

2778036

* Create data frame

In [464]:
df = pd.DataFrame(columns = ['word', 'pos', 'ner', 'clause'], data = [li][0][:])
df

,word,pos,ner,clause
0,"""",PU,O,B_CLS
1,สมชาย,NN,B_PER,I_CLS
2,"""",PU,O,I_CLS
3,ลอยลำ,VV,O,I_CLS
4,นายก,NN,O,I_CLS
...,...,...,...,...
2778031,_,PU,I_MEA,I_CLS
2778032,บาท,CL,E_MEA,I_CLS
2778033,_,PU,O,I_CLS
2778034,เป็นต้น,AV,O,E_CLS


* Recheck

In [465]:
#count
df.count()

word      2778036
pos       2714726
ner       2714726
clause    2714726
dtype: int64

In [466]:
#count word "" = sentence // True word: 2778036-63310=2714726
df.groupby([df["word"] == ""]).count()

,word,pos,ner,clause
word,,,,
False,2714726,2714726,2714726,2714726
True,63310,0,0,0


**Check Point [df1]**

In [467]:
df1=df.copy()
#try
df1

,word,pos,ner,clause
0,"""",PU,O,B_CLS
1,สมชาย,NN,B_PER,I_CLS
2,"""",PU,O,I_CLS
3,ลอยลำ,VV,O,I_CLS
4,นายก,NN,O,I_CLS
...,...,...,...,...
2778031,_,PU,I_MEA,I_CLS
2778032,บาท,CL,E_MEA,I_CLS
2778033,_,PU,O,I_CLS
2778034,เป็นต้น,AV,O,E_CLS


## Label "O"

original file have "O" > Remove "O" to "delete"

In [468]:
#original [df]
df[df["clause"] == "O"].count()

word      74079
pos       74079
ner       74079
clause    74079
dtype: int64

In [469]:
#original [df]
df[df["word"] == "_"].count()

word      402380
pos       402380
ner       402380
clause    402380
dtype: int64

In [470]:
#original [df]
df[df["word"] == ""].count()

word      63310
pos           0
ner           0
clause        0
dtype: int64

In [471]:
#copy [df1]
df1.clause[df1["clause"] == "O"] = "delete" #O=Space
df1[df1["clause"] == "delete"]

,word,pos,ner,clause
157,_,PU,O,delete
186,_,PU,O,delete
190,_,PU,O,delete
258,_,PU,O,delete
284,_,PU,O,delete
...,...,...,...,...
2777906,_,PU,O,delete
2777914,_,PU,O,delete
2777966,_,PU,O,delete
2777973,_,PU,O,delete


* label "O" in word " "

In [472]:
df1.pos[df1["word"] == ""] = "XX" #XX=other
df1.ner[df1["word"] == ""] = "O" #O=outside
df1.clause[df1["word"] == ""] = "O" #O=Space
df1.word[df1["word"] == ""] = " " # space
df1[df1["word"] == " "]

,word,pos,ner,clause
15,,XX,O,O
36,,XX,O,O
76,,XX,O,O
86,,XX,O,O
152,,XX,O,O
...,...,...,...,...
2777829,,XX,O,O
2777897,,XX,O,O
2777945,,XX,O,O
2777990,,XX,O,O


* change space word "_" to word " "

In [473]:
df1.word[df1["word"] == "_"] = " " #change word "_" to " "
#check
df1.word[df1["word"] == "_"]

Series([], Name: word, dtype: object)

In [474]:
#Cheack clause
df1.groupby("clause").count()

,word,pos,ner
clause,,,
B_CLS,215301,215301,215301
E_CLS,213247,213247,213247
I_CLS,2212099,2212099,2212099
O,63310,63310,63310
delete,74079,74079,74079


* change label "delete" to "I_CLS"

In [475]:
df1.clause[df1["clause"] == "delete"] = "I_CLS" #O=Space
#check
df1.clause[df1["clause"] == "delete"]

Series([], Name: clause, dtype: object)

In [476]:
#Cheack clause
df1.groupby("clause").count()

,word,pos,ner
clause,,,
B_CLS,215301,215301,215301
E_CLS,213247,213247,213247
I_CLS,2286178,2286178,2286178
O,63310,63310,63310


In [334]:
# df1.to_csv('Original_Clean_Train.csv')

**Check point [df_SENT]**

In [477]:
df_SENT=df1.copy()
#try
df_SENT

,word,pos,ner,clause
0,"""",PU,O,B_CLS
1,สมชาย,NN,B_PER,I_CLS
2,"""",PU,O,I_CLS
3,ลอยลำ,VV,O,I_CLS
4,นายก,NN,O,I_CLS
...,...,...,...,...
2778031,,PU,I_MEA,I_CLS
2778032,บาท,CL,E_MEA,I_CLS
2778033,,PU,O,I_CLS
2778034,เป็นต้น,AV,O,E_CLS


In [478]:
#Cheack clause
df_SENT.groupby("clause").count()

,word,pos,ner
clause,,,
B_CLS,215301,215301,215301
E_CLS,213247,213247,213247
I_CLS,2286178,2286178,2286178
O,63310,63310,63310


## Change B,I,E_CLS to I_SENT

In [479]:
df_SENT.clause[df_SENT["clause"] == "B_CLS"] = "I_SENT"
df_SENT.clause[df_SENT["clause"] == "I_CLS"] = "I_SENT"
df_SENT.clause[df_SENT["clause"] == "E_CLS"] = "I_SENT"
#try
df_SENT

,word,pos,ner,clause
0,"""",PU,O,I_SENT
1,สมชาย,NN,B_PER,I_SENT
2,"""",PU,O,I_SENT
3,ลอยลำ,VV,O,I_SENT
4,นายก,NN,O,I_SENT
...,...,...,...,...
2778031,,PU,I_MEA,I_SENT
2778032,บาท,CL,E_MEA,I_SENT
2778033,,PU,O,I_SENT
2778034,เป็นต้น,AV,O,I_SENT


In [480]:
#Cheack clause
df_SENT.groupby("clause").count()

,word,pos,ner
clause,,,
I_SENT,2714726,2714726,2714726
O,63310,63310,63310


**Check point [df_SENT_IO]**

In [482]:
df_SENT_IO=df_SENT.copy()
#try
df_SENT_IO

,word,pos,ner,clause
0,"""",PU,O,I_SENT
1,สมชาย,NN,B_PER,I_SENT
2,"""",PU,O,I_SENT
3,ลอยลำ,VV,O,I_SENT
4,นายก,NN,O,I_SENT
...,...,...,...,...
2778031,,PU,I_MEA,I_SENT
2778032,บาท,CL,E_MEA,I_SENT
2778033,,PU,O,I_SENT
2778034,เป็นต้น,AV,O,I_SENT


## Clean Data

delete 2,<=400 sentences

* Count words

In [484]:
df_SENT_IO.describe()

,word,pos,ner,clause
count,2778036,2778036,2778036,2778036
unique,48945,16,40,2
top,,NN,O,I_SENT
freq,465690,758019,2322403,2714726


* Tracking sentence

In [485]:
def convert_to_simple_transformer_format(df):
    word = []
    pos = []
    ner = []
    clause = []
    sentence_id = []
    current_sentence = 1
    
    for i in range(len(df['clause'])):
        sentence_id.append(current_sentence)
        word.append(df['word'][i])
        pos.append(df['pos'][i])
        ner.append(df['ner'][i])
        clause.append(df['clause'][i])
        if(df['clause'][i] == 'O'):
            current_sentence = current_sentence + 1

    return pd.DataFrame(
        {"word": word, "pos":pos,"ner":ner, "clause": clause,"sentence_id": sentence_id}
    )

In [486]:
df_SENT_IO2=convert_to_simple_transformer_format(df_SENT_IO)
df_SENT_IO2 #// True 63,310 sentence

,word,pos,ner,clause,sentence_id
0,"""",PU,O,I_SENT,1
1,สมชาย,NN,B_PER,I_SENT,1
2,"""",PU,O,I_SENT,1
3,ลอยลำ,VV,O,I_SENT,1
4,นายก,NN,O,I_SENT,1
...,...,...,...,...,...
2778031,,PU,I_MEA,I_SENT,63310
2778032,บาท,CL,E_MEA,I_SENT,63310
2778033,,PU,O,I_SENT,63310
2778034,เป็นต้น,AV,O,I_SENT,63310


**Check point [df_SENT_BIEO3]**

In [487]:
df_SENT_IO3=df_SENT_IO2.copy()
#try
df_SENT_IO3

,word,pos,ner,clause,sentence_id
0,"""",PU,O,I_SENT,1
1,สมชาย,NN,B_PER,I_SENT,1
2,"""",PU,O,I_SENT,1
3,ลอยลำ,VV,O,I_SENT,1
4,นายก,NN,O,I_SENT,1
...,...,...,...,...,...
2778031,,PU,I_MEA,I_SENT,63310
2778032,บาท,CL,E_MEA,I_SENT,63310
2778033,,PU,O,I_SENT,63310
2778034,เป็นต้น,AV,O,I_SENT,63310


* delete word = 2

In [488]:
#find id > word=2
count_IO3=df_SENT_IO3.groupby("sentence_id").count()
sentence_id2=count_IO3[count_IO3['word']== 2].index
sentence_id2

Int64Index([12375, 14022, 17482, 20389, 21303, 21304, 21305, 21306, 21307,
            21308,
            ...
            55204, 55206, 55207, 55210, 56800, 58635, 58719, 59366, 59367,
            62781],
           dtype='int64', name='sentence_id', length=108)

In [489]:
for i in sentence_id2:
    index_sentence_id2=df_SENT_IO3[df_SENT_IO3["sentence_id"]== i].index
    df_SENT_IO3=df_SENT_IO3.drop(index_sentence_id2)
df_SENT_IO3.reset_index(inplace=True)
df_SENT_IO3

,index,word,pos,ner,clause,sentence_id
0,0,"""",PU,O,I_SENT,1
1,1,สมชาย,NN,B_PER,I_SENT,1
2,2,"""",PU,O,I_SENT,1
3,3,ลอยลำ,VV,O,I_SENT,1
4,4,นายก,NN,O,I_SENT,1
...,...,...,...,...,...,...
2777815,2778031,,PU,I_MEA,I_SENT,63310
2777816,2778032,บาท,CL,E_MEA,I_SENT,63310
2777817,2778033,,PU,O,I_SENT,63310
2777818,2778034,เป็นต้น,AV,O,I_SENT,63310


* delete word >= 400

In [490]:
#find id > word>=400
count_IO3=df_SENT_IO3.groupby("sentence_id").count()
sentence_id400=count_IO3[count_IO3['word']>= 400].index
sentence_id400

Int64Index([62, 9810, 21214, 47132, 47133, 47336, 51119], dtype='int64', name='sentence_id')

In [491]:
for i in sentence_id400:
    index_sentence_id400=df_SENT_IO3[df_SENT_IO3["sentence_id"]== i].index
    df_SENT_IO3=df_SENT_IO3.drop(index_sentence_id400)
df_SENT_IO3.reset_index(inplace=True)
df_SENT_IO3

,level_0,index,word,pos,ner,clause,sentence_id
0,0,0,"""",PU,O,I_SENT,1
1,1,1,สมชาย,NN,B_PER,I_SENT,1
2,2,2,"""",PU,O,I_SENT,1
3,3,3,ลอยลำ,VV,O,I_SENT,1
4,4,4,นายก,NN,O,I_SENT,1
...,...,...,...,...,...,...,...
2773709,2777815,2778031,,PU,I_MEA,I_SENT,63310
2773710,2777816,2778032,บาท,CL,E_MEA,I_SENT,63310
2773711,2777817,2778033,,PU,O,I_SENT,63310
2773712,2777818,2778034,เป็นต้น,AV,O,I_SENT,63310


## Save Data_Train_CRF[IO].csv

In [492]:
df_SENT_IO4 = pd.DataFrame(columns = ['word', 'pos', 'ner', 'clause'], data = df_SENT_IO3)
df_SENT_IO4

,word,pos,ner,clause
0,"""",PU,O,I_SENT
1,สมชาย,NN,B_PER,I_SENT
2,"""",PU,O,I_SENT
3,ลอยลำ,VV,O,I_SENT
4,นายก,NN,O,I_SENT
...,...,...,...,...
2773709,,PU,I_MEA,I_SENT
2773710,บาท,CL,E_MEA,I_SENT
2773711,,PU,O,I_SENT
2773712,เป็นต้น,AV,O,I_SENT


In [493]:
df_SENT_IO4.to_csv('Data_Train_CRF[IO].csv')

# Prepare data for CRF-Cut

* Split data

In [104]:
li = []

for filename in tqdm(train):
    f = open(filename).read()
    f = list(map(lambda x:x.split('\t'),f.split('\n')))
    # print(f)
    del f[-1]
    li+=f
    #print(len(li))
    
# for filename in tqdm(val):
#     f = open(filename).read()
#     f = list(map(lambda x:x.split('\t'),f.split('\n')))
#     #print(f)
#     del f[-1]
#     li+=f
#     #print(len(li))

# for filename in tqdm(test):
#     f = open(filename).read()
#     f = list(map(lambda x:x.split('\t'),f.split('\n')))
#     #print(f)
#     del f[-1]
#     li+=f
#     #print(len(li))

100%|██████████| 3794/3794 [00:06<00:00, 582.79it/s]


In [105]:
#Try
li[0:2]

[['"', 'PU', 'O', 'B_CLS'], ['สมชาย', 'NN', 'B_PER', 'I_CLS']]

In [106]:
#Try
len(li)

2778036

* Create data frame

In [107]:
df = pd.DataFrame(columns = ['word', 'pos', 'ner', 'clause'], data = [li][0][:])
df

,word,pos,ner,clause
0,"""",PU,O,B_CLS
1,สมชาย,NN,B_PER,I_CLS
2,"""",PU,O,I_CLS
3,ลอยลำ,VV,O,I_CLS
4,นายก,NN,O,I_CLS
...,...,...,...,...
2778031,_,PU,I_MEA,I_CLS
2778032,บาท,CL,E_MEA,I_CLS
2778033,_,PU,O,I_CLS
2778034,เป็นต้น,AV,O,E_CLS


* Recheck

In [108]:
#count
df.count()

word      2778036
pos       2714726
ner       2714726
clause    2714726
dtype: int64

In [109]:
#count word "" = sentence // True word: 2778036-63310=2714726
df.groupby([df["word"] == ""]).count()

,word,pos,ner,clause
word,,,,
False,2714726,2714726,2714726,2714726
True,63310,0,0,0


**Check Point [df1]**

In [110]:
df1=df.copy()
#try
df1

,word,pos,ner,clause
0,"""",PU,O,B_CLS
1,สมชาย,NN,B_PER,I_CLS
2,"""",PU,O,I_CLS
3,ลอยลำ,VV,O,I_CLS
4,นายก,NN,O,I_CLS
...,...,...,...,...
2778031,_,PU,I_MEA,I_CLS
2778032,บาท,CL,E_MEA,I_CLS
2778033,_,PU,O,I_CLS
2778034,เป็นต้น,AV,O,E_CLS


## Label "O"

original file have "O" > Remove "O" to "delete"

In [111]:
#original [df]
df[df["clause"] == "O"].count()

word      74079
pos       74079
ner       74079
clause    74079
dtype: int64

In [112]:
#original [df]
df[df["word"] == "_"].count()

word      402380
pos       402380
ner       402380
clause    402380
dtype: int64

In [113]:
#original [df]
df[df["word"] == ""].count()

word      63310
pos           0
ner           0
clause        0
dtype: int64

In [114]:
#copy [df1]
df1.clause[df1["clause"] == "O"] = "delete" #O=Space
df1[df1["clause"] == "delete"]

,word,pos,ner,clause
157,_,PU,O,delete
186,_,PU,O,delete
190,_,PU,O,delete
258,_,PU,O,delete
284,_,PU,O,delete
...,...,...,...,...
2777906,_,PU,O,delete
2777914,_,PU,O,delete
2777966,_,PU,O,delete
2777973,_,PU,O,delete


* label "O" in word " "

In [115]:
df1.pos[df1["word"] == ""] = "XX" #XX=other
df1.ner[df1["word"] == ""] = "O" #O=outside
df1.clause[df1["word"] == ""] = "O" #O=Space
df1.word[df1["word"] == ""] = " " # space
df1[df1["word"] == " "]

,word,pos,ner,clause
15,,XX,O,O
36,,XX,O,O
76,,XX,O,O
86,,XX,O,O
152,,XX,O,O
...,...,...,...,...
2777829,,XX,O,O
2777897,,XX,O,O
2777945,,XX,O,O
2777990,,XX,O,O


* change space word "_" to word " "

In [116]:
df1.word[df1["word"] == "_"] = " " #change word "_" to " "
#check
df1.word[df1["word"] == "_"]

Series([], Name: word, dtype: object)

In [117]:
#Cheack clause
df1.groupby("clause").count()

,word,pos,ner
clause,,,
B_CLS,215301,215301,215301
E_CLS,213247,213247,213247
I_CLS,2212099,2212099,2212099
O,63310,63310,63310
delete,74079,74079,74079


* change label "delete" to "I_CLS"

In [118]:
df1.clause[df1["clause"] == "delete"] = "I_CLS" #O=Space
#check
df1.clause[df1["clause"] == "delete"]

Series([], Name: clause, dtype: object)

In [119]:
#Cheack clause
df1.groupby("clause").count()

,word,pos,ner
clause,,,
B_CLS,215301,215301,215301
E_CLS,213247,213247,213247
I_CLS,2286178,2286178,2286178
O,63310,63310,63310


In [120]:
# df1.to_csv('Original_Clean_Train.csv')

**Check point [df_SENT]**

In [121]:
df_SENT=df1.copy()
#try
df_SENT

,word,pos,ner,clause
0,"""",PU,O,B_CLS
1,สมชาย,NN,B_PER,I_CLS
2,"""",PU,O,I_CLS
3,ลอยลำ,VV,O,I_CLS
4,นายก,NN,O,I_CLS
...,...,...,...,...
2778031,,PU,I_MEA,I_CLS
2778032,บาท,CL,E_MEA,I_CLS
2778033,,PU,O,I_CLS
2778034,เป็นต้น,AV,O,E_CLS


## Change B,I,E_CLS to I_SENT

In [122]:
df_SENT.clause[df_SENT["clause"] == "B_CLS"] = "I_SENT"
df_SENT.clause[df_SENT["clause"] == "I_CLS"] = "I_SENT"
df_SENT.clause[df_SENT["clause"] == "E_CLS"] = "I_SENT"
#try
df_SENT

,word,pos,ner,clause
0,"""",PU,O,I_SENT
1,สมชาย,NN,B_PER,I_SENT
2,"""",PU,O,I_SENT
3,ลอยลำ,VV,O,I_SENT
4,นายก,NN,O,I_SENT
...,...,...,...,...
2778031,,PU,I_MEA,I_SENT
2778032,บาท,CL,E_MEA,I_SENT
2778033,,PU,O,I_SENT
2778034,เป็นต้น,AV,O,I_SENT


In [123]:
#Cheack clause
df_SENT.groupby("clause").count()

,word,pos,ner
clause,,,
I_SENT,2714726,2714726,2714726
O,63310,63310,63310


**Check point [df_SENT_IO]**

In [124]:
df_SENT_IO=df_SENT.copy()
#try
df_SENT_IO

,word,pos,ner,clause
0,"""",PU,O,I_SENT
1,สมชาย,NN,B_PER,I_SENT
2,"""",PU,O,I_SENT
3,ลอยลำ,VV,O,I_SENT
4,นายก,NN,O,I_SENT
...,...,...,...,...
2778031,,PU,I_MEA,I_SENT
2778032,บาท,CL,E_MEA,I_SENT
2778033,,PU,O,I_SENT
2778034,เป็นต้น,AV,O,I_SENT


## Clean Data

delete 2,<=400 sentences

* Count words

In [125]:
df_SENT_IO.describe()

,word,pos,ner,clause
count,2778036,2778036,2778036,2778036
unique,48945,16,40,2
top,,NN,O,I_SENT
freq,465690,758019,2322403,2714726


* Tracking sentence

In [126]:
def convert_to_simple_transformer_format(df):
    word = []
    pos = []
    ner = []
    clause = []
    sentence_id = []
    current_sentence = 1
    
    for i in tqdm(range(len(df['clause']))):
        sentence_id.append(current_sentence)
        word.append(df['word'][i])
        pos.append(df['pos'][i])
        ner.append(df['ner'][i])
        clause.append(df['clause'][i])
        if(df['clause'][i] == 'O'):
            current_sentence = current_sentence + 1

    return pd.DataFrame(
        {"word": word, "pos":pos,"ner":ner, "clause": clause,"sentence_id": sentence_id}
    )

In [127]:
df_SENT_IO2=convert_to_simple_transformer_format(df_SENT_IO)
df_SENT_IO2 #// True 63,310 sentence

100%|██████████| 2778036/2778036 [01:03<00:00, 43904.66it/s]


,word,pos,ner,clause,sentence_id
0,"""",PU,O,I_SENT,1
1,สมชาย,NN,B_PER,I_SENT,1
2,"""",PU,O,I_SENT,1
3,ลอยลำ,VV,O,I_SENT,1
4,นายก,NN,O,I_SENT,1
...,...,...,...,...,...
2778031,,PU,I_MEA,I_SENT,63310
2778032,บาท,CL,E_MEA,I_SENT,63310
2778033,,PU,O,I_SENT,63310
2778034,เป็นต้น,AV,O,I_SENT,63310


**Check point [df_SENT_BIEO3]**

In [128]:
df_SENT_IO3=df_SENT_IO2.copy()
#try
df_SENT_IO3

,word,pos,ner,clause,sentence_id
0,"""",PU,O,I_SENT,1
1,สมชาย,NN,B_PER,I_SENT,1
2,"""",PU,O,I_SENT,1
3,ลอยลำ,VV,O,I_SENT,1
4,นายก,NN,O,I_SENT,1
...,...,...,...,...,...
2778031,,PU,I_MEA,I_SENT,63310
2778032,บาท,CL,E_MEA,I_SENT,63310
2778033,,PU,O,I_SENT,63310
2778034,เป็นต้น,AV,O,I_SENT,63310


* delete word = 2

In [129]:
#find id > word=2
count_IO3=df_SENT_IO3.groupby("sentence_id").count()
sentence_id2=count_IO3[count_IO3['word']== 2].index
sentence_id2

Int64Index([12375, 14022, 17482, 20389, 21303, 21304, 21305, 21306, 21307,
            21308,
            ...
            55204, 55206, 55207, 55210, 56800, 58635, 58719, 59366, 59367,
            62781],
           dtype='int64', name='sentence_id', length=108)

In [130]:
for i in tqdm(sentence_id2):
    index_sentence_id2=df_SENT_IO3[df_SENT_IO3["sentence_id"]== i].index
    df_SENT_IO3=df_SENT_IO3.drop(index_sentence_id2)
df_SENT_IO3.reset_index(inplace=True)
df_SENT_IO3

100%|██████████| 108/108 [00:44<00:00,  2.41it/s]


,index,word,pos,ner,clause,sentence_id
0,0,"""",PU,O,I_SENT,1
1,1,สมชาย,NN,B_PER,I_SENT,1
2,2,"""",PU,O,I_SENT,1
3,3,ลอยลำ,VV,O,I_SENT,1
4,4,นายก,NN,O,I_SENT,1
...,...,...,...,...,...,...
2777815,2778031,,PU,I_MEA,I_SENT,63310
2777816,2778032,บาท,CL,E_MEA,I_SENT,63310
2777817,2778033,,PU,O,I_SENT,63310
2777818,2778034,เป็นต้น,AV,O,I_SENT,63310


* delete word >= 400

In [131]:
#find id > word>=400
count_IO3=df_SENT_IO3.groupby("sentence_id").count()
sentence_id400=count_IO3[count_IO3['word']>= 400].index
sentence_id400

Int64Index([62, 9810, 21214, 47132, 47133, 47336, 51119], dtype='int64', name='sentence_id')

In [132]:
for i in tqdm(sentence_id400):
    index_sentence_id400=df_SENT_IO3[df_SENT_IO3["sentence_id"]== i].index
    df_SENT_IO3=df_SENT_IO3.drop(index_sentence_id400)
df_SENT_IO3.reset_index(inplace=True)
df_SENT_IO3

100%|██████████| 7/7 [00:03<00:00,  2.18it/s]


,level_0,index,word,pos,ner,clause,sentence_id
0,0,0,"""",PU,O,I_SENT,1
1,1,1,สมชาย,NN,B_PER,I_SENT,1
2,2,2,"""",PU,O,I_SENT,1
3,3,3,ลอยลำ,VV,O,I_SENT,1
4,4,4,นายก,NN,O,I_SENT,1
...,...,...,...,...,...,...,...
2773709,2777815,2778031,,PU,I_MEA,I_SENT,63310
2773710,2777816,2778032,บาท,CL,E_MEA,I_SENT,63310
2773711,2777817,2778033,,PU,O,I_SENT,63310
2773712,2777818,2778034,เป็นต้น,AV,O,I_SENT,63310


## Save Data_Train_CRF[IO].csv

In [133]:
df_SENT_IO4 = pd.DataFrame(columns = ['word', 'pos', 'ner', 'clause'], data = df_SENT_IO3)
df_SENT_IO4

,word,pos,ner,clause
0,"""",PU,O,I_SENT
1,สมชาย,NN,B_PER,I_SENT
2,"""",PU,O,I_SENT
3,ลอยลำ,VV,O,I_SENT
4,นายก,NN,O,I_SENT
...,...,...,...,...
2773709,,PU,I_MEA,I_SENT
2773710,บาท,CL,E_MEA,I_SENT
2773711,,PU,O,I_SENT
2773712,เป็นต้น,AV,O,I_SENT


In [134]:
# df_SENT_IO4.to_csv('Data_Train_CRF[IO].csv')

# Submission file to .csv

## Import data

In [508]:
!gdown --id 1-75mKQIFOUgWrOJ4NZM1N-fwuBw8gWfI

Downloading...
From: https://drive.google.com/uc?id=1-75mKQIFOUgWrOJ4NZM1N-fwuBw8gWfI
To: /root/ss_test.txt
100%|████████████████████████████████████████| 844k/844k [00:00<00:00, 44.7MB/s]


In [510]:
li_sub = open('ss_test.txt').read()
li_sub = list(map(lambda x:x.split('\t'),li_sub.split('\n')))

In [514]:
li_sub[0:4]

[['กล่าว'], ['ชม'], ['ตร.'], ['เร่ง']]

In [511]:
len(li_sub)

74405

## create data frame

In [516]:
df_sub = pd.DataFrame(columns = ['word'], data = [li_sub][0][:])
df_sub

,word
0,กล่าว
1,ชม
2,ตร.
3,เร่ง
4,สำนวน
...,...
74400,การ
74401,
74402,2/2004
74403,


## Change "" to " "

In [525]:
df_sub[df_sub["word"] == " "].count()

word    11257
dtype: int64

In [523]:
df_sub.word[df_sub["word"] == ""] = " " #change word "_" to " "
#check
df_sub.word[df_sub["word"] == ""]

Series([], Name: word, dtype: object)

## save ss_test.csv

In [526]:
df_sub.to_csv('ss_test.csv')